In [1]:
from ortools.sat.cp_model_pb2 import CpSolverStatus
from tqdm import tqdm
from pyscipopt import Model, quicksum
import uuid

from ortools.sat.python.cp_model import CpModel, CpSolver

In [2]:
from loguru import logger as log

In [3]:
import numpy as np
from vrp_study.data_model import Tariff, Cargo, Node
from vrp_study.data_model import TariffCost
from vrp_study.ortools_routing_model_pdptw.pdptw_routing_manager_builder import PDRoutingManagerBuilder
from vrp_study.ortools_routing_model_pdptw.routing_model import find_optimal_paths

from dataclasses import dataclass

In [4]:
# lc1_10_8	98	42949.56	Shobb	31-mar-18

In [5]:
benchmark_type = 'pdp_100'
name = 'lc101.txt'

In [6]:
from typing import Optional

tariff = None
cargos: list[Cargo] = []
depo: Optional[Node] = None

In [7]:
id2info = {}
p2coordinates = {}
with open(f'../data/Li & Lim benchmark/{benchmark_type}/{name}', 'r') as file:
    for i, line in enumerate(file):
        line = line.split('\t')
        if i == 0:
            tariff = Tariff(
                id='car',
                capacity=int(line[1]),
                max_count=int(line[0]),
                cost_per_distance=[TariffCost(
                    min_dst_km=0,
                    max_dst_km=10000,
                    cost_per_km=1,
                    fixed_cost=0
                )]
            )
        else:
            c_id = int(line[0])
            x = int(line[1])
            y = int(line[2])

            mass = int(line[3])

            et = int(line[4])
            lt = int(line[5])
            st = int(line[6])

            pick_up = int(line[7])
            delivery = int(line[8])
            if pick_up == delivery:
                # print(12)
                depo = Node(
                    id=0,
                    cargo_id=c_id,
                    capacity=0,
                    service_time=0,
                    start_time=0,
                    end_time=lt,
                    coordinates=(x, y)
                )
                continue
            if pick_up == 0:
                if c_id not in id2info:
                    id2info[c_id] = {}
                id2info[c_id][0] = (x, y, mass, et, lt, st, c_id, delivery)
            else:
                delivery = c_id
                c_id = pick_up
                if c_id not in id2info:
                    id2info[c_id] = {}
                id2info[c_id][1] = (x, y, mass, et, lt, st, pick_up, delivery)


In [8]:
depo

Node(id=0, cargo_id=0, capacity=0, start_time=0, end_time=1236, service_time=0, coordinates=(40, 50))

In [9]:

for k, v in id2info.items():
    cargos.append(
        Cargo(
            id=k,
            nodes=[
                Node(
                    cargo_id=k,
                    id=v[i][6] if i == 0 else v[i][7],
                    capacity=v[i][2],
                    service_time=v[i][5],
                    start_time=v[i][3],
                    end_time=v[i][4],
                    coordinates=(v[i][0], v[i][1])
                )
                for i in range(2)
            ]
        )
    )

In [10]:
p2coordinates.update({
    crg.nodes[i].id: crg.nodes[i].coordinates for crg in cargos for i in range(2)
})
p2coordinates[depo.id] = depo.coordinates
distance_matrix = {(u, v): np.sqrt((du[0] - dv[0]) ** 2 + (du[1] - dv[1]) ** 2) for u, du in
                   p2coordinates.items() for
                   v, dv in p2coordinates.items()}
time_matrix = {(u, v): np.sqrt((du[0] - dv[0]) ** 2 + (du[1] - dv[1]) ** 2) for u, du in p2coordinates.items() for
               v, dv in p2coordinates.items()}

In [11]:

from vrp_study.configs import ModelConfig

routing_manager = PDRoutingManagerBuilder(
    distance_matrix=distance_matrix,
    time_matrix=time_matrix,
    model_config=ModelConfig(max_execution_time_minutes=1)
)

routing_manager.add_cargos(cargos)
routing_manager.add_tariff(tariff)

routing_manager.add_depo(depo)

# routing_manager.distance_matrix = distance_matrix
# routing_manager.time_matrix = time_matrix


routing_manager = routing_manager.build()

In [12]:
from vrp_study.ortools_routing_model_pdptw.solution_builder import SolutionBuilder

sol = find_optimal_paths(routing_manager, SolutionBuilder())[0]

2025-07-17 19:28:03.850 | INFO     | vrp_study.ortools_routing_model_pdptw.solution_builder:get_initial_solution:102 - (1, 32.074999999999996)
2025-07-17 19:28:03.851 | INFO     | vrp_study.ortools_routing_model_pdptw.solution_builder:get_initial_solution:102 - (1, 16.0875)
2025-07-17 19:28:03.852 | INFO     | vrp_study.ortools_routing_model_pdptw.solution_builder:get_initial_solution:102 - (1, 8.09375)
2025-07-17 19:28:03.852 | INFO     | vrp_study.ortools_routing_model_pdptw.solution_builder:get_initial_solution:102 - (2, 4.096875)
2025-07-17 19:28:03.853 | INFO     | vrp_study.ortools_routing_model_pdptw.solution_builder:get_initial_solution:102 - (2, 2.0984374999999997)
2025-07-17 19:28:03.854 | INFO     | vrp_study.ortools_routing_model_pdptw.solution_builder:get_initial_solution:102 - (3, 1.09921875)
2025-07-17 19:28:03.854 | INFO     | vrp_study.ortools_routing_model_pdptw.solution_builder:get_initial_solution:102 - (6, 0.599609375)
2025-07-17 19:28:03.855 | INFO     | vrp_study

53 69 False


  0%|          | 0/1 [00:00<?, ?it/s]

2025-07-17 19:28:03.863 | INFO     | vrp_study.ortools_routing_model_pdptw.routing_model:find_optimal_paths:438 - problem size: 13
2025-07-17 19:28:03.863 | INFO     | vrp_study.ortools_routing_model_pdptw.routing_model:do_solve:376 - Начало создания модели
2025-07-17 19:28:03.865 | INFO     | vrp_study.ortools_routing_model_pdptw.routing_model:add_distance_dimension:296 - Добавление размерности для расстояния
2025-07-17 19:28:03.865 | INFO     | vrp_study.ortools_routing_model_pdptw.routing_model:add_count_dimension:332 - Добавление размерности для расстояния
2025-07-17 19:28:03.865 | INFO     | vrp_study.ortools_routing_model_pdptw.routing_model:add_pick_up_and_delivery:266 - Добавление ограничения для порядка доставки
2025-07-17 19:28:03.865 | INFO     | vrp_study.ortools_routing_model_pdptw.routing_model:add_vehicles_cost:240 - Добавление стоимостей машин
2025-07-17 19:28:03.865 | INFO     | vrp_study.ortools_routing_model_pdptw.routing_model:add_time_window:191 - add time
2025-07-

  0%|          | 0/1 [00:00<?, ?it/s]

...

2025-07-17 19:28:04.806 | INFO     | vrp_study.ortools_routing_model_pdptw.routing_model:find_optimal_paths:438 - problem size: 7
2025-07-17 19:28:04.807 | INFO     | vrp_study.ortools_routing_model_pdptw.routing_model:do_solve:376 - Начало создания модели
2025-07-17 19:28:04.808 | INFO     | vrp_study.ortools_routing_model_pdptw.routing_model:add_distance_dimension:296 - Добавление размерности для расстояния
2025-07-17 19:28:04.809 | INFO     | vrp_study.ortools_routing_model_pdptw.routing_model:add_count_dimension:332 - Добавление размерности для расстояния
2025-07-17 19:28:04.810 | INFO     | vrp_study.ortools_routing_model_pdptw.routing_model:add_pick_up_and_delivery:266 - Добавление ограничения для порядка доставки
2025-07-17 19:28:04.811 | INFO     | vrp_study.ortools_routing_model_pdptw.routing_model:add_vehicles_cost:240 - Добавление стоимостей машин
2025-07-17 19:28:04.811 | INFO     | vrp_study.ortools_routing_model_pdptw.routing_model:add_time_window:191 - add time
2025-07-1

  0%|          | 0/1 [00:00<?, ?it/s]

2025-07-17 19:28:04.829 | INFO     | vrp_study.ortools_routing_model_pdptw.routing_model:find_optimal_paths:438 - problem size: 3
2025-07-17 19:28:04.829 | INFO     | vrp_study.ortools_routing_model_pdptw.routing_model:do_solve:376 - Начало создания модели
2025-07-17 19:28:04.831 | INFO     | vrp_study.ortools_routing_model_pdptw.routing_model:add_distance_dimension:296 - Добавление размерности для расстояния
2025-07-17 19:28:04.832 | INFO     | vrp_study.ortools_routing_model_pdptw.routing_model:add_count_dimension:332 - Добавление размерности для расстояния
2025-07-17 19:28:04.832 | INFO     | vrp_study.ortools_routing_model_pdptw.routing_model:add_pick_up_and_delivery:266 - Добавление ограничения для порядка доставки
2025-07-17 19:28:04.833 | INFO     | vrp_study.ortools_routing_model_pdptw.routing_model:add_vehicles_cost:240 - Добавление стоимостей машин
2025-07-17 19:28:04.833 | INFO     | vrp_study.ortools_routing_model_pdptw.routing_model:add_time_window:191 - add time
2025-07-1

  0%|          | 0/1 [00:00<?, ?it/s]

2025-07-17 19:28:05.284 | INFO     | vrp_study.ortools_routing_model_pdptw.routing_model:find_optimal_paths:438 - problem size: 3
2025-07-17 19:28:05.285 | INFO     | vrp_study.ortools_routing_model_pdptw.routing_model:do_solve:376 - Начало создания модели
2025-07-17 19:28:05.285 | INFO     | vrp_study.ortools_routing_model_pdptw.routing_model:add_distance_dimension:296 - Добавление размерности для расстояния
2025-07-17 19:28:05.286 | INFO     | vrp_study.ortools_routing_model_pdptw.routing_model:add_count_dimension:332 - Добавление размерности для расстояния
2025-07-17 19:28:05.286 | INFO     | vrp_study.ortools_routing_model_pdptw.routing_model:add_pick_up_and_delivery:266 - Добавление ограничения для порядка доставки
2025-07-17 19:28:05.286 | INFO     | vrp_study.ortools_routing_model_pdptw.routing_model:add_vehicles_cost:240 - Добавление стоимостей машин
2025-07-17 19:28:05.287 | INFO     | vrp_study.ortools_routing_model_pdptw.routing_model:add_time_window:191 - add time
2025-07-1

In [13]:
# sol

In [14]:
@dataclass
class Route:
    cost: int
    path: list[int]
    path_set: Optional[set[int]] = None

    def __post_init__(self):
        self.path_set = set(self.path)

In [15]:
class SelectModelCpSat:
    def __init__(self, routes: list[Route], nodes: set[int]):
        self.model = CpModel()
        self.routes = routes
        self.Z = {}
        self.nodes = nodes
        self._build()

    def _build(self):
        routes = self.routes
        Z = self.Z
        model = self.model
        for i, r in enumerate(routes):
            Z[i] = model.new_bool_var(name=f'Z_{i}')

        for n in self.nodes:
            model.add(sum(Z[i] for i, route in enumerate(routes) if n in route.path_set) >= 1)

    def optimize(self, solver: CpSolver) -> Optional[list[Route]]:
        self.model.clear_objective()
        self.model.minimize(sum(self.Z[i] * r.cost for i, r in enumerate(self.routes)))

        status = solver.solve(self.model)

        log.info(f'model solve with status: {status}')

        if status == CpSolverStatus.OPTIMAL or status == CpSolverStatus.FEASIBLE:
            return [r for i, r in enumerate(self.routes) if solver.Value(self.Z[i]) == 1]
        return None

In [16]:
import pyscipopt


class SelectModelLP:
    def __init__(self, routes: list[Route], nodes: set[int]):
        self.model = Model(f"Simple_LP_Problem_{uuid.uuid4()}")
        self.routes = routes
        self.Z = {}
        self.nodes = nodes
        self._build()

    def _build(self):
        routes = self.routes
        Z = self.Z
        model = self.model

        for i, r in enumerate(routes):
            Z[i] = model.addVar(name=f'Z_{i}', lb=0, ub=1, vtype='C')

        for n in self.nodes:
            model.addCons(quicksum(Z[i] for i, route in enumerate(routes) if n in route.path_set) == 1, name=f'my_{n}')

    def get_dual_variables(self) -> Optional[list[float]]:
        model = self.model
        model.setObjective(quicksum(self.Z[i] * r.cost for i, r in enumerate(self.routes)), 'minimize')
        
        model.setParam('presolving/maxrounds', 0)
        model.setParam('lp/solvefreq', 1)
        model.setParam('display/verblevel', 0)  # Suppress output
        model.setHeuristics(pyscipopt.SCIP_PARAMSETTING.OFF)
        model.disablePropagation()

        model.optimize()

        if model.getStatus() == 'optimal':
            # for k, v in self.Z.items():
            #     print(k, model.getVal(v))
            result = {}
            for cons in model.getConss():
                name = cons.name
                if 'my' in name:
                    dual_value = model.getDualSolVal(cons)
                    idx = int(name.split('_')[1])
                    result[idx] = dual_value
            return result
        return None

In [17]:
NODES = set(n.id for n in routing_manager.nodes() if n.is_transit)

In [18]:
routes: list[Route] = []
for nodes in routing_manager.get_pick_up_and_delivery_nodes():
    q = [routing_manager.get_depo_index()] + nodes + [routing_manager.get_depo_index()]
    routes.append(
        Route(
            cost=int(
                sum(routing_manager.get_distance(
                    routing_manager.nodes()[q[i]],
                    routing_manager.nodes()[q[i + 1]]
                ) for i in range(len(q) - 1))
            ),
            path=nodes
        )
    )

# for nodes in sol:
#     if len(nodes) == 0:
#         continue
#     routes.append(
#         Route(
#             cost=int(
#                 sum(routing_manager.get_distance(
#                     routing_manager.nodes()[nodes[i]],
#                     routing_manager.nodes()[nodes[i + 1]]
#                 ) for i in range(len(nodes) - 1))
#             ),
#             path=nodes[1:-1]
#         )
#     )

In [19]:
len(NODES)

106

In [20]:
sum(r.cost for r in routes)

3329

In [21]:
lp = SelectModelLP(routes=routes, nodes=NODES)
res = lp.get_dual_variables()

In [22]:
res

{1: 48.0,
 2: 48.0,
 3: 44.0,
 4: 44.0,
 5: 35.0,
 6: 35.0,
 7: 42.0,
 8: 42.0,
 9: 33.0,
 10: 33.0,
 11: 38.0,
 12: 38.0,
 13: 87.0,
 14: 87.0,
 15: 68.0,
 16: 68.0,
 17: 81.0,
 18: 81.0,
 19: 80.0,
 20: 80.0,
 21: 30.0,
 22: 30.0,
 23: 41.0,
 24: 41.0,
 25: 35.0,
 26: 35.0,
 27: 26.0,
 28: 26.0,
 29: 34.0,
 30: 34.0,
 31: 42.0,
 32: 42.0,
 33: 70.0,
 34: 70.0,
 35: 78.0,
 36: 78.0,
 37: 83.0,
 38: 83.0,
 39: 89.0,
 40: 89.0,
 41: 70.0,
 42: 70.0,
 43: 42.0,
 44: 42.0,
 45: 38.0,
 46: 38.0,
 47: 44.0,
 48: 44.0,
 49: 47.0,
 50: 47.0,
 51: 39.0,
 52: 39.0,
 53: 47.0,
 54: 47.0,
 55: 50.0,
 56: 50.0,
 57: 96.0,
 58: 96.0,
 59: 93.0,
 60: 93.0,
 61: 72.0,
 62: 72.0,
 63: 90.0,
 64: 90.0,
 65: 45.0,
 66: 45.0,
 67: 45.0,
 68: 45.0,
 69: 39.0,
 70: 39.0,
 71: 43.0,
 72: 43.0,
 73: 47.0,
 74: 47.0,
 75: 34.0,
 76: 34.0,
 77: 117.0,
 78: 117.0,
 79: 117.0,
 80: 117.0,
 81: 113.0,
 82: 113.0,
 83: 100.0,
 84: 100.0,
 85: 107.0,
 86: 107.0,
 87: 73.0,
 88: 73.0,
 89: 64.0,
 90: 64.0,
 91: 62.0

In [23]:
from vrp_study.routing_manager import RoutingManager


class PDPTWModel:
    def __init__(self, routing_manager: RoutingManager, cost: dict[float], ignore_routes: list[Route]):
        self.routing_manager = routing_manager
        self.cost = cost

        self.model = CpModel()
        self.X = {}
        self.T = {}
        self.Q = {}
        self.ignore_routes = ignore_routes
        self._build()

    def _build(self):
        routing_manager = self.routing_manager
        log.info(f'problem size: {len(routing_manager.nodes())}')

        N = len(routing_manager.nodes())
        M = 1_000_000

        X = self.X
        T = self.T
        Q = self.Q

        model = self.model
        depo = routing_manager.get_depo_index()

        for i in range(N):
            for j in range(N):
                if i == j:
                    X[i, j] = model.new_constant(0)
                else:
                    X[i, j] = model.new_bool_var(f'x_{i, j}')

        max_time = max(n.end_time for n in routing_manager.nodes())
        max_mass = max(car.capacity for car in routing_manager.cars())

        log.debug(f'max_time: {max_time}, max_mass: {max_mass}')
        for i in range(N):
            T[i] = model.new_int_var(lb=0, ub=max_time * 2, name=f'time_{i}')
            Q[i] = model.new_int_var(lb=0, ub=max_mass, name=f'mass_{i}')

        model.add(sum(X[depo, i] for i in range(N)) == 1)

        for i, node in enumerate(routing_manager.nodes()):
            model.add(
                sum(X[i, j] for j in range(N)) == sum(X[j, i] for j in range(N))
            )
            #   
            if i == 0:
                continue
            model.add(sum(X[i,j] for j in range(N)) <= 1)
            for j in range(N):
                if i == j:
                    continue
                ni = routing_manager.nodes()[i]
                nj = routing_manager.nodes()[j]

                model.add(
                    T[i] + int(routing_manager.get_time(ni, nj) + routing_manager.nodes()[i].service_time) <=
                    T[j] + M * (1 - X[i, j]))

                model.add(
                    Q[i] + routing_manager.nodes()[i].demand <=
                    Q[j] + M * (1 - X[i, j]))

        for node_a, node_b in routing_manager.get_pick_up_and_delivery_nodes():
            a, b = node_a, node_b
            model.add(T[a] <= T[b])
            model.add(sum(X[k, a] + X[k, b] for k in range(N)) == 2 * sum(X[k, a] for k in range(N)))
        # 
        for i, node in enumerate(routing_manager.nodes()):
            model.add(node.start_time <= T[i])
            model.add(T[i] <= node.end_time)

        for route in self.ignore_routes:
            route = route.path
            v = X[0, route[0]] + X[route[-1], 0]
            for i in range(len(route) - 1):
                v += X[route[i], route[i + 1]]
            model.add(v != len(route) + 1)

    def ignore_path(self, route: list[int]):
        model = self.model
        X = self.X
        
        v = X[0, route[0]] + X[route[-1], 0]
        for i in range(len(route) - 1):
            v += X[route[i], route[i + 1]]
        model.add(v != len(route) + 1)

    def optimize(self, solver: CpSolver):
        self.model.clear_objective()
        N = len(self.routing_manager.nodes())
        r = self.routing_manager
        obj = sum(self.X[i, j] * int(routing_manager.get_distance(
            r.nodes()[i], r.nodes()[j])) for i in range(N) for j in range(N)) - sum(
            self.cost[i] * sum(self.X[j, i] for j in range(N)) for i in range(1, N))
        self.model.minimize(obj)

        status = solver.solve(self.model)

        log.info(f'model solve with status: {status}')

        if status == CpSolverStatus.OPTIMAL or status == CpSolverStatus.FEASIBLE:
            start = 0
            path = []
            while True:
                next_node = next(iter(n for n in range(N) if solver.Value(self.X[start, n]) == 1))
                if next_node == 0:
                    if path[0] == 0:
                        path = path[1:]
                    return path
                else:
                    start = next_node
                    path.append(start)

        #     return [r for i, r in enumerate(self.routes) if solver.Value(self.Z[i]) == 1]
        # return None

In [24]:
from ortools.sat import sat_parameters_pb2

best = {'preferred_variable_order': 2,
        'clause_cleanup_protection': 1,
        'max_presolve_iterations': 5,
        'cp_model_probing_level': 1,
        'presolve_probing_deterministic_time_limit': 10.0,
        'search_branching': 2,
        'feasibility_jump_linearization_level': 0,
        'fp_rounding': 0,
        'polish_lp_solution': True,
        'linearization_level': 0,
        'cut_level': 2,
        'max_all_diff_cut_size': 128,
        'symmetry_level': 0,
        'num_workers': 32}


def get_solver():
    solver = CpSolver()

    for k, v in best.items():
        if isinstance(v, list):
            for ss in v:
                solver.parameters.ignore_subsolvers.append(ss)
        else:
            if 'ignore_subsolvers' in k:
                if v:
                    solver.parameters.ignore_subsolvers.append(k.split(':')[1])
            else:
                exec(f'solver.parameters.{k} = {v}')
    # solver.parameters.use_lns = True
    # solver.parameters.lns_num_threads = 4
    solver.parameters.log_search_progress = False
    solver.parameters.max_time_in_seconds = 60.0 * 10

    # packing_subsolver = sat_parameters_pb2.SatParameters()
    # packing_subsolver.name = "MyPackingSubsolver"
    # packing_subsolver.use_area_energetic_reasoning_in_no_overlap_2d = False
    # packing_subsolver.use_energetic_reasoning_in_no_overlap_2d = False
    # packing_subsolver.use_timetabling_in_no_overlap_2d = False
    # packing_subsolver.max_pairs_pairwise_reasoning_in_no_overlap_2d = 5_000
    # packing_subsolver.
    # # Add the subsolver to the portfolio
    # solver.parameters.subsolver_params.append(packing_subsolver)  # Define the subsolver
    # solver.parameters.extra_subsolvers.append(
    #     packing_subsolver.name
    # )  # Activate the subsolver

    return solver

In [25]:
solver = get_solver()

In [26]:
routes

[Route(cost=48, path=[1, 2], path_set={1, 2}),
 Route(cost=44, path=[3, 4], path_set={3, 4}),
 Route(cost=35, path=[5, 6], path_set={5, 6}),
 Route(cost=42, path=[7, 8], path_set={8, 7}),
 Route(cost=33, path=[9, 10], path_set={9, 10}),
 Route(cost=38, path=[11, 12], path_set={11, 12}),
 Route(cost=87, path=[13, 14], path_set={13, 14}),
 Route(cost=68, path=[15, 16], path_set={16, 15}),
 Route(cost=81, path=[17, 18], path_set={17, 18}),
 Route(cost=80, path=[19, 20], path_set={19, 20}),
 Route(cost=30, path=[21, 22], path_set={21, 22}),
 Route(cost=41, path=[23, 24], path_set={24, 23}),
 Route(cost=35, path=[25, 26], path_set={25, 26}),
 Route(cost=26, path=[27, 28], path_set={27, 28}),
 Route(cost=34, path=[29, 30], path_set={29, 30}),
 Route(cost=42, path=[31, 32], path_set={32, 31}),
 Route(cost=70, path=[33, 34], path_set={33, 34}),
 Route(cost=78, path=[35, 36], path_set={35, 36}),
 Route(cost=83, path=[37, 38], path_set={37, 38}),
 Route(cost=89, path=[39, 40], path_set={40, 39})

In [27]:
pdptw_model = PDPTWModel(routing_manager, cost=res, ignore_routes=routes)
path = pdptw_model.optimize(solver)
path

2025-07-17 19:28:06.328 | INFO     | __main__:_build:18 - problem size: 107
2025-07-17 19:28:06.358 | DEBUG    | __main__:_build:40 - max_time: 1236, max_mass: 200
2025-07-17 19:28:08.805 | INFO     | __main__:optimize:105 - model solve with status: 4


[77, 83, 84, 81, 79, 78, 82, 80, 85, 86, 51, 52]

In [28]:
cp_select_model = SelectModelCpSat(routes, NODES)
routes = cp_select_model.optimize(solver)

2025-07-17 19:28:08.813 | INFO     | __main__:optimize:25 - model solve with status: 4


In [29]:
sum(r.cost for r in routes)

3329

In [30]:
from tqdm.notebook import trange

pdptw_model = PDPTWModel(routing_manager, cost=res, ignore_routes=routes)
set_paths = set(tuple(r.path) for r in routes )
for _ in trange(500):
    lp = SelectModelLP(routes=routes, nodes=NODES)
    res = lp.get_dual_variables()
    for _ in range(1):
        path = pdptw_model.optimize(solver)
        if path is None or solver.objective_value > 0:
            break
        assert tuple(path) not in set_paths
        set_paths.add(tuple(path))
        pdptw_model.ignore_path(path)
        log.info(f'{path}; {solver.objective_value}')
        nodes = [0] + path + [0]
        routes.append(Route(
            cost=int(
                sum(routing_manager.get_distance(
                    routing_manager.nodes()[nodes[i]],
                    routing_manager.nodes()[nodes[i + 1]]
                ) for i in range(len(nodes) - 1))
            ),
            path=path
        ))

2025-07-17 19:28:08.823 | INFO     | __main__:_build:18 - problem size: 107
2025-07-17 19:28:08.886 | DEBUG    | __main__:_build:40 - max_time: 1236, max_mass: 200


  0%|          | 0/500 [00:00<?, ?it/s]

2025-07-17 19:28:11.173 | INFO     | __main__:optimize:105 - model solve with status: 4
2025-07-17 19:28:11.174 | INFO     | __main__:<module>:15 - [77, 83, 84, 81, 79, 78, 82, 80, 85, 86, 51, 52]; -1034.0
2025-07-17 19:28:13.283 | INFO     | __main__:optimize:105 - model solve with status: 4
2025-07-17 19:28:13.285 | INFO     | __main__:<module>:15 - [77, 83, 84, 81, 79, 78, 82, 80, 85, 86]; -982.0
2025-07-17 19:28:15.414 | INFO     | __main__:optimize:105 - model solve with status: 4
2025-07-17 19:28:15.416 | INFO     | __main__:<module>:15 - [77, 83, 84, 81, 79, 78, 82, 80, 55, 56, 53, 54]; -930.0
2025-07-17 19:28:17.689 | INFO     | __main__:optimize:105 - model solve with status: 4
2025-07-17 19:28:17.691 | INFO     | __main__:<module>:15 - [77, 83, 84, 81, 79, 78, 82, 80, 55, 56, 51, 52]; -915.0
2025-07-17 19:28:19.857 | INFO     | __main__:optimize:105 - model solve with status: 4
2025-07-17 19:28:19.859 | INFO     | __main__:<module>:15 - [77, 83, 84, 81, 79, 78, 82, 80, 53, 54

In [31]:
cp_select_model = SelectModelCpSat(routes, NODES)
routes = cp_select_model.optimize(solver)

2025-07-17 20:36:21.977 | INFO     | __main__:optimize:25 - model solve with status: 4


In [32]:
sum(r.cost for r in routes)

1235